In [ ]:
# Install required packages first
import os
import subprocess
pip_commands = [
    ["pip", "install","-q", "--extra-index-url", "https://download.pytorch.org/whl/cu117", 
     "torch==2.0.0", "torchvision==0.15.0", "omegaconf", "torchmetrics==0.10.3", 
     "fvcore", "iopath", "xformers==0.0.18", "submitit", "numpy<2.0"],
    ["pip", "install", "-q",  "--extra-index-url", "https://pypi.nvidia.com", "cuml-cu11"],
    ["pip", "install","-q",  "black==22.6.0", "flake8==5.0.4", "pylint==2.15.0"],
    ["pip", "install", "-q", "mmsegmentation==0.27.0"],
    ["pip", "install","-q", "mmcv-full==1.5.0"],
    ["pip", "install","-q", "nibabel"]
]

for cmd in pip_commands:
    try:
        print(cmd)
        subprocess.run(cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Failed to install packages with command: {cmd}")
        print(f"Error: {e}")

['pip', 'install', '-q', '--extra-index-url', 'https://download.pytorch.org/whl/cu117', 'torch==2.0.0', 'torchvision==0.15.0', 'omegaconf', 'torchmetrics==0.10.3', 'fvcore', 'iopath', 'xformers==0.0.18', 'submitit', 'numpy<2.0']
['pip', 'install', '-q', '--extra-index-url', 'https://pypi.nvidia.com', 'cuml-cu11']


In [1]:
!nvidia-smi

Fri May 23 13:35:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  |   00000000:B7:00.0 Off |                    0 |
| N/A   42C    P0             59W /  400W |       3MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
# ═════════════  CONFIG  ════════════════════════════════════════════════
CHECKPOINT_DIR = "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100/output_ddp_p16_23_1150/checkpoints"
BASE_CKPT       = f"{CHECKPOINT_DIR}/base_model_epoch_19.pt"  #base_model_final.pt   # or base_model_epoch_X.pt
AGG_CKPT        = f"{CHECKPOINT_DIR}/aggregator_epoch_19.pt"  #aggregator_final.pt   # or aggregator_epoch_X.pt
ROOT_DIR  = "/rsrch7/home/ip_rsrch/wulab/HuiXu/testData/Projects/SABR/SABR_Dataset_Final_V1"
# ───────────────────────────────────────────────────────────────────────

import os, sys, torch, nibabel as nib, numpy as np, pandas as pd
from tqdm.auto import tqdm
import torch.nn as nn

# --- 1. CombinedModel ---------------------------------------------------------
sys.path.insert(0, "/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6")
from dinov2.models.vision_transformer import DinoVisionTransformer

def build_backbone():
    m = DinoVisionTransformer(
        img_size=448, patch_size=16, drop_path_rate=0.0,
        block_chunks=1, drop_path_uniform=True,
        embed_dim=768, depth=12, num_heads=12, mlp_ratio=4,
        num_register_tokens=5, init_values=1e-5,
    )
    # materialise empty tokens
    with torch.no_grad():
        for n in ("cls_token","register_tokens","mask_token"):
            if getattr(m,n).storage().size()==0:
                real = torch.zeros_like(getattr(m,n))
                torch.nn.init.normal_(real,std=0.02)
                setattr(m,n,torch.nn.Parameter(real,requires_grad=True))
    return m

class CombinedModel(nn.Module):
    def __init__(self, base_model, num_attn_heads=3, num_layers=2,
                 hidden_dim=1024, chunk_feat_dim=768, dropout_rate=0.3, lse_tau=1.0):
        super().__init__()
        self.base = base_model
        self.lse_tau = lse_tau
        self.chunk_feat_dim = chunk_feat_dim + 3                # 768 + spacing(3) = 771

        enc = nn.TransformerEncoderLayer(
            d_model=self.chunk_feat_dim, nhead=num_attn_heads,
            dim_feedforward=hidden_dim, dropout=dropout_rate, batch_first=True)
        self.transformer = nn.TransformerEncoder(enc, num_layers=num_layers)
        self.classifier  = nn.Linear(self.chunk_feat_dim, 6)    # not used here

    def _chunk_embed(self, chunk):
        # backbone returns [1, 768]
        with torch.no_grad():          # frozen backbone for inference
            return self.base(chunk.unsqueeze(0))

    def pooled(self, x, spacing=(1.0,1.0,1.0)):
        S, device, dtype = x.size(0), x.device, x.dtype
        feats = torch.cat([ self._chunk_embed(x[i]) for i in range(S) ], dim=0)   # [S,768]
        spacing_vec = torch.tensor(spacing,dtype=dtype,device=device).expand(S,3)
        feats = torch.cat([feats, spacing_vec], dim=1)                            # [S,771]
        enc = self.transformer(feats.unsqueeze(0))                                # [1,S,771]
        return self.lse_tau * torch.logsumexp(enc / self.lse_tau, dim=1).squeeze(0)  # [771]

# build
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CombinedModel(build_backbone()).to(device).eval()

# --- 2. load weights ----------------------------------------------------------
missing, _ = model.base.load_state_dict(torch.load(BASE_CKPT, map_location="cpu"), strict=False)
agg_state  = torch.load(AGG_CKPT, map_location="cpu")
# drop backbone keys if present
agg_state  = {k:v for k,v in agg_state.items() if not k.startswith("base.")}
model.load_state_dict(agg_state, strict=False)
print(f"Loaded weights • backbone missing={len(missing)} • agg keys={len(agg_state)}")

# --- 3. volume → batch of chunks ---------------------------------------------
CHUNK_DEPTH = 3; V_MIN, V_MAX = -1000, 150; OUT_SZ = (448,448)
@torch.no_grad()
def patient_vector_771(nii_path):
    """
    Return one 771-D pooled vector for a CT whose in-plane size is 512×512.
    Skip (return None) if that condition isn’t met.
    """
    hdr  = nib.load(nii_path)
    vol  = hdr.get_fdata().astype(np.float32)                # numpy array

    # --- quick shape guard ---------------------------------------------------
    if vol.shape[0] != 512 or vol.shape[1] != 512:
        print(f"⚠️  {nii_path} skipped – in-plane dims {vol.shape[:2]} not 512×512")
        return None                                          # skip this CT
    # ------------------------------------------------------------------------

    # window + scale to [0,1]
    vol = np.clip((vol - V_MIN) / (V_MAX - V_MIN), 0, 1)     # shape (512,512,D)
    D   = vol.shape[2]
    S   = D // CHUNK_DEPTH
    if S == 0:
        print(f"⚠️  {nii_path} has <{CHUNK_DEPTH} slices"); return None

    # --- build [S,3,448,448] tensor -----------------------------------------
    S = max(S, 72)
    chunks = []
    for start in range(0, D - CHUNK_DEPTH + 1, CHUNK_DEPTH):
        chunk = vol[:, :, start:start + CHUNK_DEPTH]    # exact 3 slices (512,512,3)
        chunk = chunk.transpose(2, 0, 1)                # → (3,512,512)
    
        t = torch.from_numpy(chunk).unsqueeze(0)        # [1,3,H,W]
        t = torch.nn.functional.interpolate(
                t, OUT_SZ, mode="bilinear", align_corners=False)
        t = (t - 0.5) / 0.5
        chunks.append(t)

    x = torch.cat(chunks).to(device)                         # [S,3,448,448]
    spacing = hdr.header.get_zooms()[:3]
    vec = model.pooled(x, spacing).cpu().numpy()             # [771]

    return vec



# --- 4. iterate folders -------------------------------------------------------
folders = [os.path.join(ROOT_DIR,p) for p in os.listdir(ROOT_DIR)
           if os.path.isdir(os.path.join(ROOT_DIR,p))]
print(f"▶ Found {len(folders)} folders")
folders.sort()
folders = folders[1142+263:]
for fdir in tqdm(folders):
    nii_path = os.path.join(fdir, "CT.nii.gz")
    if not os.path.exists(nii_path):
        print("⚠️ ", nii_path, " missing"); continue

    vec = patient_vector_771(nii_path)
    if vec is None:
        print("⚠️ ", fdir, " had <3 slices"); continue

    pd.DataFrame([vec], columns=[f"feat_{i}" for i in range(771)]
                 ).to_csv(os.path.join(
                     fdir, f"{os.path.basename(fdir)}_aggMori.csv"),
                     index=False)




/opt/venv/jupyter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")
/tmp/ipykernel_443/1877029792.py:26: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if

Loaded weights • backbone missing=0 • agg keys=26
▶ Found 1428 folders


100%|██████████| 286/286 [18:24<00:00,  3.86s/it]


In [3]:
!nvidia-smi

Fri May 23 13:54:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  |   00000000:B7:00.0 Off |                    0 |
| N/A   45C    P0             89W /  400W |    3529MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----